<a href="https://colab.research.google.com/github/GenaroHacker/actionable/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
#@title Set Up
!git clone https://github.com/GenaroHacker/write_imports.git
!git clone https://github.com/GenaroHacker/dict_to_kivy_app.git

from write_imports.write_imports import write_imports



In [4]:



# Delete this line to see the new imports
%%capture



#@title Import Statements
#Modules: ['write_imports', 'dict_to_kivy_app']



from write_imports.write_imports import write_imports

from write_imports.LearningPython.BasicExamples.intro_functions import print_hello_world

from dict_to_kivy_app.modules.write_imports import write_imports

from dict_to_kivy_app.modules.create_kv import create_kv

from dict_to_kivy_app.modules.create_app import create_app

from dict_to_kivy_app.modules.create_main import create_main

from dict_to_kivy_app.modules.create_screen_and_kv import create_screen_and_kv

from dict_to_kivy_app.modules.create_screen import sanitize_function_name
from dict_to_kivy_app.modules.create_screen import create_screen

from dict_to_kivy_app.modules.create_utils import create_utils





print(write_imports([]))






In [5]:
# @title Create App

# Define app structure dictionary
screens = {
    'task_selector': {
        'reachable_screens': ['subtask_sequence'],
        'custom_functions': ['t1', 't2', 't3', 't4', 't5', 'Start'],
        'grid_columns': 1,
        'color': (0, 0.9, 0.5),
    },
    'subtask_sequence': {
        'reachable_screens': ['task_selector'],
        'custom_functions': ['Pause', 'Cancel', 'Next Step'],
        'grid_columns': 3,
        'color': (1, 1, 1),
    }
}

# Create and download app
create_app(
    screen_dict=screens,
    folder_name="example_apps",
    app_name="SampleApp",
    title_height=100,
    button_height=50,
    navigation_columns=2,
    transition='NoTransition',
    compress_and_download=True
)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'example_apps/SampleApp'